# Introduction to functional programming

In Python, functions are first class objects — you can pass them around like other objects.

Sometimes it's convenient to think in terms of `map()` and `reduce()`. This pattern is fundamental in data analytics and some packages, such as `pandas`. 

A couple of very common functions, `sorted()` and `filter()`, always works with a function as one of their arguments. It can be confusing the first time you see it.

With no functional argument, `sorted` does what you'd expect:

In [1]:
l = [0.001, 1, 3, 51, 41 , 601]
sorted(l)

[0.001, 1, 3, 41, 51, 601]

What if we want to sort based on the number of characters? Don't ask why, we just do. Then we write a function that returns a key which, when sorted, will give the ordering we want.

In [2]:
def strlen(n):
    return len(str(n))

In [3]:
sorted(l, key=strlen)

[1, 3, 51, 41, 601, 0.001]

We could rewrite that tiny function as a lambda, which is basically a little unnamed function:

In [4]:
sorted(l, key=lambda n: len(str(n)))

[1, 3, 51, 41, 601, 0.001]

Note that there's often already a function; if we didn't have to cast the floats to strings first, we could do:

In [5]:
m = ['one', 'two', 'three', 'four', 'five', 'six']
sorted(m, key=len)

['one', 'two', 'six', 'four', 'five', 'three']

When would you make a named function vs a lambda? It all depends on whether you want to use it again or not. 

### `map` and `lambda`

You can think of `map` as 'apply this function to all of these items'. 

In [6]:
def sq(n):
    return n**2

# In Python 3, map produces an iterator, not a list.
# So we must cast to list to inspect its contents.
# This is good for your code, but not for this demo.
list(map(sq, l))

[1e-06, 1, 9, 2601, 1681, 361201]

We can get around defining that tiny function `sq()` with a lambda, which you can think of as a temporary, 'throwaway' function:

In [6]:
list(map(lambda n: n**2, l))

[1e-06, 1, 9, 2601, 1681, 361201]

## Currying with `partial`

We could also use Python's built-in `pow` command, but it takes two arguments. So this won't work:

In [13]:
pow(2, 3)

8

In [14]:
list(map(pow(2), l))

TypeError: pow expected at least 2 arguments, got 1

Instead we can use another functional programming concept, currying. To do this, we use `functools.partial`. There's a catch, though... you can only apply `partial` to the first argument. So for the purposes of this demo:

In [15]:
import math  # To keep our namespace vaguely clean.
pow = lambda a, b: math.pow(b, a)
pow(2, 3)

9.0

In [16]:
from functools import partial
list(map(partial(pow, 2), l))

[1e-06, 1.0, 9.0, 2601.0, 1681.0, 361201.0]

## List comprehension is often better though

In practice, we'd often write this as a list comprehension. Then we can skip the creation of the funciton or lambda entirely:

In [17]:
[n**2 for n in l]

[1e-06, 1, 9, 2601, 1681, 361201]

One of the advantages of `map` is that it is 'lazy', so if you map a function to a giant list, you don't get a giant list back, you get an iterator. A list-comp would give you a giant list, possibly jamming the memory on your box. 

If we wanted to do something with these numbers, we can actually skip the list formation another way — using a **generator expression**. This is worth doing if we only want to step over that list once, and don't need to exploit any special `list` methods:

In [18]:
for i in (n**2 for n in l):
    print(i)

1e-06
1
9
2601
1681
361201


[Read about the difference.](http://stackoverflow.com/questions/47789/generator-expressions-vs-list-comprehension)

### `reduce`

`reduce` takes a sequence and applies some function to it recursively. You could think of it like a running sum, say, but for any function, not just summing.

In [19]:
def runsum(a, b):
    return a + b

# For some reason reduce is not in the main namespace like map
from functools import reduce

reduce(runsum, l)

697.001

In [21]:
def runmult(a, b):
    return a * b

reduce(runmult, l)

3770.073

### `partial` for making curry

'Preapplying' an argument to a function is sometimes useful. For example, we might have a general function for raising a number *a* to the power *b*, but then want another function which raises numbers to the 3rd power.

I could do this by simply calling the first function from the second:

In [22]:
def power(a, b):
    return a**b

def cuber(a):
    return power(a, 3)

cuber(2)

8

But some people might find it more inuitive to do it this way:

In [14]:
from functools import partial

cuber = partial(power, b=3)

cuber(2)

8

In [23]:
pow?

This is called 'currying' the function `power()`. 

## Closures

A closure is a nested function, in which the inner function uses variables passed only to the outer function.

[For example:](http://www.shutupandship.com/2012/01/python-closures-explained.html)

In [4]:
def generate_power_func(n):
    def nth_power(x):
        return x**n
    return nth_power

This function makes functions. It takes the power `n`, and returns a function that takes one argument: the number you want to raise to the power that is now 'baked in' to the function you created.

In [9]:
square = generate_power_func(2)

In [10]:
square(4)

16

It's so baked in that we can delete the outer function from the namespace:

In [11]:
del generate_power_func

In [12]:
cube = generate_power_func(3)

NameError: name 'generate_power_func' is not defined

We just deleted it, so we can't call it anymore. We can still call the function we made though:

In [13]:
square(9)

81

That's closure!

<hr />

## Taking it further

* Try following along with [this excellent tutorial](https://maryrosecook.com/blog/post/a-practical-introduction-to-functional-programming)
* Learn Clojure or Haskell! Or even JavaScript, which uses closure often if it's written well.

<hr />

<div>
<img src="https://avatars1.githubusercontent.com/u/1692321?s=50"><p style="text-align:center">© Agile Geoscience 2016</p>
</div>